In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from collections import Counter

import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import yaml


from tensorflow.keras import models, layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Reshape


In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [ ]:
dataset_root = "FridgeDetection_data"
splits = ["train", "test", "valid"]

def load_data(split):
    image_dir = os.path.join(dataset_root, split, "images")
    label_dir = os.path.join(dataset_root, split, "labelTxt")
    
    images, labels = [], []
    for file in os.listdir(image_dir):
        if file.lower().endswith((".jpg", ".png")):
            img_path = os.path.join(image_dir, file)
            label_path = os.path.join(label_dir, os.path.splitext(file)[0] + ".txt")
            
            images.append(cv2.imread(img_path))
            labels.append(open(label_path).read() if os.path.exists(label_path) else None)
    
    return images, labels

data = {split: dict(zip(["images", "labels"], load_data(split))) for split in splits}

# Print summary
for split in splits:
    print(f"{split.capitalize()}: Loaded {len(data[split]['images'])} images and {len(data[split]['labels'])} labels.")


In [ ]:
def plot_images_with_labels(images, labels, title, num_images=10, max_columns=2):
    combined = list(zip(images, labels))
    random.shuffle(combined)
    images, labels = zip(*combined[:num_images])

    num_rows = (len(images) + max_columns - 1) // max_columns
    fig, axes = plt.subplots(num_rows, max_columns, figsize=(10, 5 * num_rows))
    axes = axes.flatten() if num_images > 1 else [axes]

    for ax, img, label_text in zip(axes, images, labels):
        img = img.copy()
        if label_text:
            for line in label_text.strip().splitlines():
                parts = line.split()
                if len(parts) >= 9:
                    coords = list(map(int, parts[:8]))
                    label = parts[8]
                    pts = np.array(coords).reshape((4, 2))
                    cv2.polylines(img, [pts], isClosed=True, color=(0, 0, 255), thickness=2)
                    cv2.putText(img, label, tuple(pts[0]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.axis("off")

    # Hide unused subplots
    for ax in axes[len(images):]:
        ax.axis("off")

    fig.suptitle(title, fontsize=16)
    plt.subplots_adjust(top=0.92, hspace=0.3)
    plt.show()

# Example usage
plot_images_with_labels(data['train']['images'], data['train']['labels'], "Train Set with Labels")
plot_images_with_labels(data['valid']['images'], data['valid']['labels'], "Valid Set with Labels")
plot_images_with_labels(data['test']['images'], data['test']['labels'], "Test Set with Labels")


In [ ]:
# Function to count labels
def count_labels(data):
    label_counts = Counter()
    for split in data:
        for label_data in data[split]['labels']:
            if label_data:
                for line in label_data.splitlines():
                    parts = line.split()
                    if len(parts) >= 9:
                        label = parts[8]
                        label_counts[label] += 1
    return label_counts

# Count labels across all splits
label_counts = count_labels(data)

# Print the counts
for label, count in label_counts.items():
    print(f"{label}: {count}")

In [ ]:
# Check the size of the first image in the training set
image_shape = data['train']['images'][0].shape
print(f"Image size: {image_shape}")

In [ ]:
X_train = data['train']['images']
X_val = data['valid']['images']
y_train = data['train']['labels']
y_val = data['valid']['labels']
X_test = data['test']['images']
y_test = data['test']['labels']

In [ ]:
def resize(img):
    # Convert to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Normalize pixel values to [0, 1]
    img = img.astype("float32") / 255.0
    # Clip values to ensure they are within the valid range [0, 1] for floats
    img = np.clip(img, 0.0, 1.0)
    # Resize image
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    return img

# Apply preprocessing to all sets
X_train = np.array([resize(img) for img in X_train])
X_val = np.array([resize(img) for img in X_val])
X_test = np.array([resize(img) for img in X_test])



In [ ]:
image_shape = X_train[0].shape
print(f"Image size: {image_shape}")

In [ ]:
def resize_labels(labels, original_size, new_size):
    resized_labels = []
    scale_x = new_size[0] / original_size[0]
    scale_y = new_size[1] / original_size[1]
    
    for label_data in labels:
        if label_data:
            resized_label_data = []
            for line in label_data.splitlines():
                parts = line.split()
                if len(parts) >= 9:
                    x1, y1, x2, y2, x3, y3, x4, y4, label = parts[:9]
                    coords = [int(int(p) * scale_x if i % 2 == 0 else int(p) * scale_y) for i, p in enumerate([x1, y1, x2, y2, x3, y3, x4, y4])]
                    resized_label_data.append(f"{coords[0]} {coords[1]} {coords[2]} {coords[3]} {coords[4]} {coords[5]} {coords[6]} {coords[7]} {label}")
            resized_labels.append("\n".join(resized_label_data))
        else:
            resized_labels.append(None)
    
    return resized_labels# Example usage

original_size = (640, 640)  # Assuming original images are 640x640
new_size = (224, 224)  # Resized images are 224x224

# Resize labels for train, validation, and test sets
resized_train_labels = resize_labels(y_train, original_size, new_size)
resized_val_labels = resize_labels(y_val, original_size, new_size)
resized_test_labels = resize_labels(y_test, original_size, new_size)

# Print a few examples to verify
print("Original label:", y_train[0])
print("Resized label:", resized_train_labels[0])


In [ ]:
def plot_images_with_labels(images, labels, title, num_images=10, max_columns=2):
    # Shuffle images and labels together
    combined = list(zip(images, labels))
    random.shuffle(combined)
    images, labels = zip(*combined)
    
    num_rows = (min(num_images, len(images)) + max_columns - 1) // max_columns
    plt.figure(figsize=(10, 5 * num_rows))
    
    for i in range(min(num_images, len(images))):
        img = images[i].copy()  # Make a copy of the image to avoid modifying the original image
        if labels[i]:
            for line in labels[i].splitlines():
                parts = line.split()
                if len(parts) >= 9:
                    # Extract the coordinates and label
                    x1, y1, x2, y2, x3, y3, x4, y4, label = parts[:9]
                    
                    # Convert the coordinates to integers
                    x1, y1, x2, y2, x3, y3, x4, y4 = map(int, [x1, y1, x2, y2, x3, y3, x4, y4])
                    
                    # Draw the label text
                    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                    
                    # Draw the bounding box polygon
                    cv2.polylines(img, [np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], np.int32)], 
                                   isClosed=True, color=(0, 0, 255), thickness=2)
                
        
        # Plot the image
        plt.subplot(num_rows, max_columns, i + 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert from BGR to RGB for displaying
        plt.axis("off")
    
    # Set title and show the plot
    plt.suptitle(title)
    plt.show()

# Plot 10 random images from the train set with labels
plot_images_with_labels(X_train, resized_train_labels, "Train Set with Labels")

# Plot 10 random images from the validation set with labels
plot_images_with_labels(X_val, resized_val_labels, "Valid Set with Labels")

y_train = resized_train_labels
y_val = resized_val_labels
y_test = resized_test_labels

In [ ]:
# Preprocess labels into multi-hot encoded format
def encode_labels(labels, label_counts):
    label_to_index = {label: idx for idx, label in enumerate(label_counts.keys())}
    num_classes = len(label_counts)
    
    encoded_labels = []
    for label_data in labels:
        multi_hot = np.zeros(num_classes, dtype=np.float32)
        if label_data:
            for line in label_data.splitlines():
                parts = line.split()
                if len(parts) >= 9:
                    label = parts[8]
                    if label in label_to_index:
                        multi_hot[label_to_index[label]] = 1.0
        encoded_labels.append(multi_hot)
    return np.array(encoded_labels)

# Apply preprocessing to train, validation, and test labels
y_train_encoded = encode_labels(y_train, label_counts)
y_val_encoded = encode_labels(y_val, label_counts)
y_test_encoded = encode_labels(y_test, label_counts)

In [ ]:
X_train 
X_val
X_test

y_train
y_val
y_test

y_train_encoded
y_val_encoded
y_test_encoded

In [ ]:
def get_classes_from_labels(dataset_root, splits=['train', 'valid', 'test']):
    classes = set()
    for split in splits:
        labelTxt_dir = os.path.join(dataset_root, split, "labelTxt")
        if not os.path.exists(labelTxt_dir):
            continue
        for label_file in os.listdir(labelTxt_dir):
            if not label_file.endswith('.txt'):
                continue
            label_path = os.path.join(labelTxt_dir, label_file)
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 9:
                        continue
                    class_name = parts[8]
                    classes.add(class_name)
    return sorted(classes)

def polygon_to_yolo(line, img_width, img_height, class_to_id):
    parts = line.strip().split()
    coords = list(map(int, parts[:8]))  # 8 coords: x1 y1 x2 y2 x3 y3 x4 y4
    class_name = parts[8]
    xs = coords[0::2]
    ys = coords[1::2]
    xmin, xmax = min(xs), max(xs)
    ymin, ymax = min(ys), max(ys)
    x_center = (xmin + xmax) / 2 / img_width
    y_center = (ymin + ymax) / 2 / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height
    class_id = class_to_id[class_name]
    return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

def convert_labels(dataset_root, split, class_to_id):
    images_dir = os.path.join(dataset_root, split, "images")
    labelTxt_dir = os.path.join(dataset_root, split, "labelTxt")
    labels_dir = os.path.join(dataset_root, split, "labels")

    os.makedirs(labels_dir, exist_ok=True)

    for img_file in os.listdir(images_dir):
        if not img_file.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue
        img_path = os.path.join(images_dir, img_file)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: could not read image {img_path}, skipping.")
            continue
        h, w = img.shape[:2]

        label_txt_file = os.path.splitext(img_file)[0] + ".txt"
        labelTxt_path = os.path.join(labelTxt_dir, label_txt_file)
        label_output_path = os.path.join(labels_dir, label_txt_file)

        if not os.path.exists(labelTxt_path):
            print(f"Label file not found for {img_file}, skipping.")
            continue

        with open(labelTxt_path, 'r') as f_in, open(label_output_path, 'w') as f_out:
            for line in f_in:
                yolo_line = polygon_to_yolo(line, w, h, class_to_id)
                f_out.write(yolo_line + '\n')

def main():
    dataset_root = os.path.abspath("FridgeDetection_data")

    print("Scanning labelTxt files for classes...")
    class_names = get_classes_from_labels(dataset_root)
    print(f"Classes found: {class_names}")

    class_to_id = {cls: idx for idx, cls in enumerate(class_names)}

    print("Converting labels to YOLO format...")
    for split in ['train', 'valid', 'test']:
        convert_labels(dataset_root, split, class_to_id)
        print(f"Converted {split} labels.")

    # Create data.yaml file
    data_yaml = {
        'train': os.path.join(dataset_root, 'train', 'images'),
        'val': os.path.join(dataset_root, 'valid', 'images'),
        'test': os.path.join(dataset_root, 'test', 'images'),
        'nc': len(class_names),
        'names': class_names
    }

    yaml_path = os.path.join(dataset_root, "data.yaml")
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f)

    print(f"data.yaml created at {yaml_path}")

if __name__ == "__main__":
    main()


In [ ]:

dataset_root = os.path.abspath("FridgeDetection_data")
yaml_path = os.path.join(dataset_root, "data.yaml")

# Create empty lists to store metrics
epochs = []
box_losses = []
cls_losses = []
dfl_losses = []
precisions = []
recalls = []
map50s = []
map5095s = []

# Load your YOLO model
model = YOLO('yolov8n.pt')  # or your custom model path

results = model.train(
    data='FridgeDetection_data/data.yaml',
    epochs=10,
    batch=16,
    device='0',
    project='runs',    # folder to save results
    name='exp1',       # experiment subfolder
    exist_ok=True      # overwrite if exists
)

import pandas as pd

# Now that training is done, check for the results directory
runs_dir = os.path.join("runs", "exp1")

results_csv = os.path.join(runs_dir, "results.csv")
if not os.path.isfile(results_csv):
    raise FileNotFoundError(f"'{results_csv}' not found. Make sure training has completed successfully.")

df = pd.read_csv(results_csv)

# Print columns to debug
print("Available columns in results.csv:", df.columns.tolist())

# Use correct column names based on what is available
# Common YOLOv8 columns: 'epoch', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', etc.
epochs = df['epoch'] + 1 if 'epoch' in df.columns else df.index + 1

box_losses = df['train/box_loss'] if 'train/box_loss' in df.columns else None
cls_losses = df['train/cls_loss'] if 'train/cls_loss' in df.columns else None
dfl_losses = df['train/dfl_loss'] if 'train/dfl_loss' in df.columns else None
precisions = df['metrics/precision(B)'] if 'metrics/precision(B)' in df.columns else None
recalls = df['metrics/recall(B)'] if 'metrics/recall(B)' in df.columns else None
map50s = df['metrics/mAP_0.5(B)'] if 'metrics/mAP_0.5(B)' in df.columns else None
map5095s = df['metrics/mAP_0.5:0.95(B)'] if 'metrics/mAP_0.5:0.95(B)' in df.columns else None


# Plot metrics after training
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
if box_losses is not None:
    plt.plot(epochs, box_losses, label='Box Loss')
if cls_losses is not None:
    plt.plot(epochs, cls_losses, label='Class Loss')
if dfl_losses is not None:
    plt.plot(epochs, dfl_losses, label='DFL Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training Losses')

plt.subplot(1, 2, 2)
if precisions is not None:
    plt.plot(epochs, precisions, label='Precision')
if recalls is not None:
    plt.plot(epochs, recalls, label='Recall')
if map50s is not None:
    plt.plot(epochs, map50s, label='mAP@0.5')
if map5095s is not None:
    plt.plot(epochs, map5095s, label='mAP@0.5:0.95')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()
plt.title('Validation Metrics')

plt.tight_layout()
plt.show()

# Validate the model on the validation set
results_val = model.val(data=yaml_path, imgsz=640, batch=16, workers=0, device='0')
print("Validation results:", results_val)


In [ ]:
# import os

# runs_root = 'runs'
# if os.path.exists(runs_root):
#     print("Contents of 'runs' folder:")
#     print(os.listdir(runs_root))
# else:
#     print("'runs' folder does not exist.")



In [ ]:
# from ultralytics import YOLO
# import os


# # Path to the data.yaml file
# yaml_path = "FridgeDetection_data/data.yaml"  # Path to your data.yaml file

# # Test set images folder from the data.yaml (Ensure this matches your YAML file setup)
# test_images_path = "FridgeDetection_data/test/images"  # Path to your test images

# # List all test images
# test_images = [os.path.join(test_images_path, img) for img in os.listdir(test_images_path) if img.endswith(('.jpg', '.png'))]

# # Run predictions on the test images
# for image_path in test_images:
#     # Make predictions on each test image
#     results = model.predict(image_path, device='cuda:0')  # Run on GPU 0 if available, else use CPU

#     # results is a list; iterate through each result
#     for result in results:
#         # Show predictions on the image
#         result.show()  # This will show the image with predicted bounding boxes

#         # Optionally save predictions (with bounding boxes) to disk
#         result.save()  # Save the images with predictions in the 'runs/detect' folder

#         # Optionally print details of the results
#         print(f"Predictions for {image_path}:")
#         print(result.to_df())  # Print bounding box results in a pandas dataframe format


In [ ]:
from ultralytics import YOLO
import os
import pandas as pd
from IPython.display import display

# Path to the data.yaml file (if needed)
yaml_path = "FridgeDetection_data/data.yaml"  # Path to your data.yaml file

# Test set images folder from the data.yaml (Ensure this matches your YAML file setup)
test_images_path = "FridgeDetection_data/test/images"  # Path to your test images

# List all test images
test_images = [os.path.join(test_images_path, img) for img in os.listdir(test_images_path) if img.endswith(('.jpg', '.png'))]

# Run predictions on the test images
for image_path in test_images:
    # Make predictions on each test image
    results = model.predict(image_path, device='cuda:0')  # Run on GPU 0 if available, else use CPU
    
    # results is a list; iterate through each result
    for result in results:
        # Show predictions on the image
        # result.show()  # This will show the image with predicted bounding boxes
        
        # # Optionally save predictions (with bounding boxes) to disk
        # result.save()  # Save the images with predictions in the 'runs/detect' folder

        # Optionally print details of the results
        print(f"Predictions for {image_path}:")
        
        # Convert to a pandas DataFrame
        df = result.to_df()  # Get bounding box results as a pandas DataFrame
        
        # Display the dataframe in the notebook
        display(df)  # This will print the DataFrame in the notebook output
        
        # This will print the class labels and confidence score for each detected object
        print("Predicted classes and their confidence scores:")
        if not df.empty and 'name' in df.columns and 'confidence' in df.columns:
            display(df[['name', 'confidence']])  # Display only the class and confidence columns
        else:
            print("No detections.")


In [ ]:
import random
from ultralytics import YOLO
import os
import pandas as pd
from IPython.display import display
import numpy as np

yaml_path = "FridgeDetection_data/data.yaml"  # Path to your data.yaml file

test_images_path = "FridgeDetection_data/test/images"
annotations_path = "FridgeDetection_data/test/labels"

# List all test images
all_test_images = [os.path.join(test_images_path, img) for img in os.listdir(test_images_path) if img.endswith(('.jpg', '.png'))]

# Select 10 random images
test_images = random.sample(all_test_images, min(10, len(all_test_images)))  # Ensure we don’t exceed the available count


def get_ground_truth(image_path):
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    gt_file = os.path.join(annotations_path, base_name + '.txt')
    if os.path.exists(gt_file):
        with open(gt_file, 'r') as f:
            gt_data = [line.strip().split() for line in f.readlines()]
            return pd.DataFrame(gt_data, columns=['class_id', 'x_center', 'y_center', 'width', 'height'])
    else:
        return pd.DataFrame()

def iou(box1, box2):
    x1_min = box1[0] - box1[2] / 2
    x1_max = box1[0] + box1[2] / 2
    y1_min = box1[1] - box1[3] / 2
    y1_max = box1[1] + box1[3] / 2
    
    x2_min = box2[0] - box2[2] / 2
    x2_max = box2[0] + box2[2] / 2
    y2_min = box2[1] - box2[3] / 2
    y2_max = box2[1] + box2[3] / 2
    
    inter_x_min = max(x1_min, x2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_min = max(y1_min, y2_min)
    inter_y_max = min(y1_max, y2_max)
    
    intersection_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    union_area = box1_area + box2_area - intersection_area
    return intersection_area / union_area if union_area > 0 else 0

# Run predictions and evaluation
for image_path in test_images:
    print(f"\nProcessing image: {image_path}")
    results = model.predict(image_path, device='cuda:0')

    for result in results:
        df_predictions = result.to_df()
        if not df_predictions.empty:
            print("Prediction columns:", df_predictions.columns.tolist())
        df_ground_truth = get_ground_truth(image_path)
        if not df_ground_truth.empty:
            for col in ['x_center', 'y_center', 'width', 'height']:
                df_ground_truth[col] = df_ground_truth[col].astype(float)

        if not df_predictions.empty and not df_ground_truth.empty:
            print(f"Comparing predictions with actual for {image_path}:")
            correct_predictions = 0
            total_predictions = len(df_predictions)
            for _, pred_row in df_predictions.iterrows():
                predicted_class = str(pred_row['class'])  # class id is stored as int/float
                if 'box' in df_predictions.columns:
                    box = pred_row['box']
                    if isinstance(box, (list, np.ndarray, tuple)) and len(box) == 4:
                        try:
                            x1, y1, x2, y2 = map(float, box)
                        except Exception as e:
                            print(f"Skipping invalid box data: {box} ({e})")
                            continue
                        x_center = (x1 + x2) / 2
                        y_center = (y1 + y2) / 2
                        width = x2 - x1
                        height = y2 - y1
                    else:
                        print(f"Invalid box format: {box}")
                        continue
                else:
                    print("No bounding box columns found!")
                    continue

                predicted_box = np.array([x_center, y_center, width, height])
                predicted_confidence = pred_row['confidence']
                matching_gt = df_ground_truth[df_ground_truth['class_id'] == predicted_class]

                if not matching_gt.empty:
                    gt_box = matching_gt[['x_center', 'y_center', 'width', 'height']].iloc[0].to_numpy()
                    iou_score = iou(predicted_box, gt_box)
                    if iou_score >= 0.5 and predicted_confidence > 0.5:
                        correct_predictions += 1

            accuracy = (correct_predictions / total_predictions) * 100 if total_predictions > 0 else 0
            print(f"Prediction accuracy for {image_path}: {accuracy:.2f}%")
        else:
            print("No detections or ground truth data.")



In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


In [ ]:
import os
import optuna
from ultralytics import YOLO

def objective(trial):
    # Define the fixed settings
    fixed_params = {
        "data": "FridgeDetection_data/data.yaml",
        "epochs": 10,  # use fewer epochs for fast tuning
        "imgsz": 640,
        "device": "0",
        "batch": 16,
        "workers": 0
    }

    # Define the search space
    params = {
        "lr0": trial.suggest_float("lr0", 1e-4, 1e-2, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True),
        "mosaic": trial.suggest_float("mosaic", 0.5, 1.0),
        "mixup": trial.suggest_float("mixup", 0.0, 0.3),
        "hsv_h": trial.suggest_float("hsv_h", 0.0, 0.05),
        "box": trial.suggest_float("box", 5.0, 10.0),
        "cls": trial.suggest_float("cls", 0.1, 1.0),
        "dfl": trial.suggest_float("dfl", 1.0, 3.0),
    }

    full_config = {**fixed_params, **params}

    # Load model
    model = YOLO("yolov8n.pt")

    # Train model
    model.train(**full_config)

    # Validate model
    results = model.val(data=fixed_params["data"])

    # Use mAP@0.5 as the optimization target
    map50 = results.box.map50
    return map50

# Run Optuna optimization
def tune_yolo(trials=20):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=trials)

    print("\n✅ Best hyperparameters:")
    for k, v in study.best_params.items():
        print(f"{k}: {v:.6f}")
    print(f"🏆 Best mAP@0.5: {study.best_value:.4f}")

    return study

# Run the tuner
study = tune_yolo(trials=10)  # You can increase trials for better results
